In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import sys



In [8]:
import unitTypes

unitTextVectorizationLayer = keras.layers.TextVectorization(vocabulary=unitTypes.allUnits)


next_layer = keras.layers.Dense(256**3,activation="relu")(next_layer)

In [ ]:


dataset = keras.preprocessing.image_dataset_from_directory(
    directory="D:/Downloads/Images/69000-20221206T023428Z-001/69000",
    label_mode=None, image_size=(img_width, img_height), batch_size=32,
    shuffle=True, seed=None, validation_split=None,
).map(lambda x: x / 255.0)

latent_dim = 128
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(16 * 16 * 128),
        layers.Reshape((16, 16, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)